## 1. Install Dependencies

In [1]:
!pip install -q transformers==4.49.0 datasets==3.3.2 accelerate==1.4.0 bitsandbytes==0.45.3 trl==0.15.2 peft==0.14.0 sentencepiece protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


## 2. Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Import Libraries and Check GPU

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

# Check GPU
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("PyTorch version:", torch.__version__)

CUDA available: True
GPU: Tesla T4
PyTorch version: 2.9.0+cu126


## 4. Load and Prepare Dataset

In [4]:
# Load datasets from Google Drive
print("Loading datasets from Google Drive...")
dataset = load_dataset("json", data_files={
    "train": "/content/drive/MyDrive/phishing_project/train_gemma.jsonl",
    "test": "/content/drive/MyDrive/phishing_project/eval_gemma.jsonl"
})

print(f"Train samples: {len(dataset['train'])}")
print(f"Eval samples: {len(dataset['test'])}")

# Show sample
print("\nSample data:")
print(dataset['train'][0])

Loading datasets from Google Drive...


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train samples: 14904
Eval samples: 1863

Sample data:
{'instruction': "You are a security model. Classify the following email as 'phishing' or 'safe'. Reply with exactly one word: phishing or safe.", 'input': "ena and ees industrial sic codes interoffice memo from : john lavorato & dave delainey to : all of ena and ees re : ena and ees industrial sic codes as enron continues to build its origination and customer business in the us , it is becoming increasingly important that both ena and ees target the appropriate industries and customers with our respective value propositions . double coverage and duplication waste resources and damages enron ' s reputation in the market place . it is also important that the right product , solution and sales technique are utilized to ensure the maximum benefit for our customers and enron . consistent with the past , ees will be focused on those sic codes or industries that are less energy intensive and are less sophisticated in their management of en

## 5. Convert to Conversational Format

In [5]:
def create_conversation(sample):
    """
    Convert JSONL sample to conversational format for SFTTrainer.
    SFTTrainer expects 'messages' format with role-based conversation.
    """
    return {
        "messages": [
            {"role": "user", "content": f"{sample['instruction']}\n\n{sample['input']}"},
            {"role": "assistant", "content": sample['output']}
        ]
    }

# Convert datasets
print("Converting to conversational format...")
dataset = dataset.map(create_conversation, remove_columns=list(dataset["train"].features), batched=False)

print("\nConverted sample:")
print(dataset['train'][0])

Converting to conversational format...


Map:   0%|          | 0/14904 [00:00<?, ? examples/s]

Map:   0%|          | 0/1863 [00:00<?, ? examples/s]


Converted sample:
{'messages': [{'content': "You are a security model. Classify the following email as 'phishing' or 'safe'. Reply with exactly one word: phishing or safe.\n\nena and ees industrial sic codes interoffice memo from : john lavorato & dave delainey to : all of ena and ees re : ena and ees industrial sic codes as enron continues to build its origination and customer business in the us , it is becoming increasingly important that both ena and ees target the appropriate industries and customers with our respective value propositions . double coverage and duplication waste resources and damages enron ' s reputation in the market place . it is also important that the right product , solution and sales technique are utilized to ensure the maximum benefit for our customers and enron . consistent with the past , ees will be focused on those sic codes or industries that are less energy intensive and are less sophisticated in their management of energy . ena shall continue to targe

## 6. Configure Quantization (QLoRA)

In [6]:
# 4-bit quantization config for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print("✓ QLoRA configuration ready")

✓ QLoRA configuration ready


In [7]:
from huggingface_hub import login

# Login to HuggingFace (you'll be prompted for your token)
login()

## 7. Load Tokenizer

**Note:** Using instruction-tuned tokenizer which has chat template support

In [8]:
print("Loading tokenizer from HuggingFace...")
tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-2b-it",  # Instruction-tuned tokenizer with chat template
    trust_remote_code=True,
)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✓ Tokenizer loaded")

Loading tokenizer from HuggingFace...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

✓ Tokenizer loaded


## 8. Load Model

**Downloads ~5GB - takes ~30 seconds on Colab**

In [9]:
print("Loading Gemma-2-2b model from HuggingFace...")
print("This will download ~5GB (takes ~30 seconds on Colab)")

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",  # Downloads automatically
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
)

print("✓ Model loaded successfully")

Loading Gemma-2-2b model from HuggingFace...
This will download ~5GB (takes ~30 seconds on Colab)


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

✓ Model loaded successfully


## 9. Configure LoRA

In [10]:
# LoRA configuration (as per Google's Gemma guide)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",  # Google's recommendation for Gemma
    task_type="CAUSAL_LM",
)

print("✓ LoRA configuration ready")

✓ LoRA configuration ready


## 10. Configure Training

**Training Parameters:**
- Epochs: 3
- Batch size: 4 (effective 16 with gradient accumulation)
- Learning rate: 2e-4
- Optimizer: paged_adamw_8bit (memory efficient)
- **Results saved to Google Drive** for persistence

In [11]:
# Training arguments
training_args = SFTConfig(
    output_dir="/content/drive/MyDrive/phishing_project/gemma-2-2b-phishing",  # Save to Drive!
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Can use larger batch on Colab T4
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to=[],
    max_seq_length=512,
)

print("✓ Training configuration ready")
print(f"Output will be saved to: {training_args.output_dir}")

✓ Training configuration ready
Output will be saved to: /content/drive/MyDrive/phishing_project/gemma-2-2b-phishing


## 11. Create Trainer

In [12]:
print("Setting up SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer,
)

print("✓ Trainer ready")

Setting up SFTTrainer...


Converting train dataset to ChatML:   0%|          | 0/14904 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/14904 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14904 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/14904 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✓ Trainer ready


## 12. Train Model

**This will take ~2-3 hours on T4 GPU**

**Important:** The model will be saved to Google Drive automatically, so even if Colab disconnects, your progress is safe!

In [13]:
print("🚀 Starting training...")
print("Estimated time: 2-3 hours for 3 epochs")
print("Model checkpoints will be saved to Google Drive")
print()

trainer.train()

print("\n✅ Training complete!")

🚀 Starting training...
Estimated time: 2-3 hours for 3 epochs
Model checkpoints will be saved to Google Drive



`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.91 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.30 GiB is free. Process 15544 has 12.44 GiB memory in use. Of the allocated memory 10.68 GiB is allocated by PyTorch, and 1.63 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 13. Save Final Model

In [ ]:
print("Saving final model to Google Drive...")
output_dir = "/content/drive/MyDrive/phishing_project/gemma-2-2b-phishing-final"

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ Model saved to: {output_dir}")
print("\n📥 To use locally:")
print("1. Download the 'gemma-2-2b-phishing-final' folder from Google Drive")
print("2. Extract to 'artifacts/models/gemma-2-2b-phishing' in your local project")
print("3. Load with: AutoModelForCausalLM.from_pretrained('artifacts/models/gemma-2-2b-phishing')")

## 14. Quick Test (Optional)

In [ ]:
# Test the trained model
from transformers import pipeline

# Load the trained model
pipe = pipeline(
    "text-generation",
    model=output_dir,
    device_map="auto",
)

# Test sample
test_prompt = """You are a security model. Classify the following email as 'phishing' or 'safe'. Reply with exactly one word: phishing or safe.

Subject: Verify your account now!
From: support@paypa1.com
Body: Your account has been suspended. Click here to verify immediately.

Classification:"""

result = pipe(test_prompt, max_new_tokens=10, temperature=0.1)
print("\nTest Result:")
print(result[0]['generated_text'])

## Summary

**Training Complete!**

Your fine-tuned Gemma-2-2b model is saved in Google Drive at:
- Checkpoints: `/MyDrive/phishing_project/gemma-2-2b-phishing/`
- Final model: `/MyDrive/phishing_project/gemma-2-2b-phishing-final/`

**Next Steps:**
1. Download the model folder from Google Drive
2. Use it in your local ensemble with DeBERTa
3. Evaluate performance and create ensemble metrics

**Benefits over local training:**
- ✅ 2-3 hours vs 12-15 hours locally
- ✅ Free GPU (T4)
- ✅ Persistent storage in Google Drive
- ✅ No local environment conflicts